In [2]:
pip install pycocotools-windows

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pycocotools-windows in c:\users\lant\appdata\local\programs\python\python37\lib\site-packages (2.0.0.2)


You should consider upgrading via the 'c:\users\lant\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [3]:
from __future__ import annotations
import numpy as np
from PIL import Image
import requests
from pycocotools.coco import COCO
import json
import  os
import pandas as pd

annotation_dir = "C:/Users/lant/Desktop/illegal_poacher_detection/annotations"
images_dir = "C:/Users/lant/Desktop/illegal_poacher_detection/images"
file_annotation = "instances_train2017.json"
NS =200 # download 200 atmost samples from coco dataset
def main():
    coco_annotation_file_path = os.path.join(annotation_dir, file_annotation)
    coco_annotation = COCO(annotation_file=coco_annotation_file_path)

    out_file = os.path.join(annotation_dir,file_annotation[:-5] + '_train.csv')
    out = open(out_file, 'w')
    out.write('file_name, width, height,id,xmin,ymin,xmax,ymax,class,text_label\n')

    cat_names =['dog','elephant','bear','backpack','umbrella','tiger','rhino','ranger','poacher']

    # Category Name -> Category ID.

    query_ids = coco_annotation.getCatIds(catNms=cat_names)
    cat_name_ids = dict([(k, v) for k, v in zip(query_ids, cat_names) ])
    image_ids = []
    for j, q_ids in enumerate (query_ids):
        # Get the ID of all the images containing the object of the category.
        img_ids = coco_annotation.getImgIds(catIds=[q_ids])
        print(f"Number of Images Containing {cat_names[j]}: {len(img_ids)}")
        N =0
        for img_id in img_ids:
            img_info = coco_annotation.loadImgs([img_id])[0]
            img_url = img_info["coco_url"]
            # Get all the annotations for the specified image.
            ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
            anns = coco_annotation.loadAnns(ann_ids)
            if len(anns)>0:
                for i in range(len(anns)):
                    ann = anns[i]
                    hi = img_info["height"]
                    wi = img_info["width"]
                    f_name =img_info["file_name"]
                    x1 = ann['bbox'][0]
                    x2 = ann['bbox'][0] + ann['bbox'][2]
                    y1 = ann['bbox'][1]
                    y2 = ann['bbox'][1] + ann['bbox'][3]
                    label = j+1 #ann['category_id']
                    text_label =cat_names[j] 
                    out.write('{},{},{},{},{},{},{},{},{},{}\n'.format(f_name, hi, wi, img_id, x1, y1, x2, y2, label, text_label))

            else:
                hi = img_info["height"]
                wi = img_info["width"]
                f_name =img_info["file_name"]
                out.write('{},{},{},{},{},{},{},{},{},{}\n'.format(f_name, hi, wi, img_id, -1, -1, -1, -1, -1, -1))

            # Use URL to load image.
            im = requests.get(img_url, stream=True)
            im = requests.get(img_url, stream=True).content
            with open(images_dir +"/"+ img_info['file_name'], 'wb') as handler:
                handler.write(im)
            N +=1
            image_ids.append(img_id)
            if N ==NS:
                break
    out.close()
     # Sort file by image id
    df = pd.read_csv(out_file)
    df.sort_values('id', inplace=True)
    df.to_csv(out_file, index=False)


    
main()
# if __name__ == "__main__":
    
   

   
    

loading annotations into memory...
Done (t=1676.71s)
creating index...
index created!
Number of Images Containing dog: 4385
Number of Images Containing elephant: 2143
Number of Images Containing bear: 960
Number of Images Containing backpack: 5528
Number of Images Containing umbrella: 3968
